In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from auxiliary import values as v
from auxiliary.data import imaging
from auxiliary.utils.timer import LoadingBar

from nuclei_segmentation import my_cellpose
from nuclei_segmentation.quality_control.model_tester import ModelTester

GPU activated: True


In [2]:
# v.data_path = '/mnt/c/Users/ignac/OneDrive/Nacho/CNIC/TFM/Data/'

img_path = (
        v.data_path 
        + 'SegQA/RawImages/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop.nii.gz'
)

Grid search into preprocessing steps 

In [3]:
from itertools import permutations, chain


# Cellpose params
steps_type = ['2D', '3D']
steps_thr = list(np.round(np.linspace(.2, .6, 6), 1))
steps_prob_thr = list(np.round(np.linspace(0, .5, 5), 1))

# Preprocessing steps
normalization_methods = ['norm_percentile']
pre_steps_top = ['isotropy']
pre_steps_bottom = [
    'bilateral', 
    'anisodiff'
] 

pre_steps_permutations = list(chain(*[
    permutations(pre_steps_bottom, r) 
    for r in range(1, len(pre_steps_bottom) + 1)
])) + [()]

# Postprocessing steps
post_steps_2d = [
    'merge_connected_components',
    'merge_graph',
    'clean_boundaries_opening',
]

post_steps_permutations_2d = list(chain(*[
    permutations(post_steps_2d, r) 
    for r in range(1, len(post_steps_2d) + 1)
])) + [()]

post_steps_3d = [
    'split',
    'clean_boundaries_opening',
    'clean_boundaries_closing',
]

post_steps_permutations_3d = list(chain(*[
    permutations(post_steps_3d, r) 
    for r in range(1, len(post_steps_3d) + 1)
])) + [()]

pipelines_dict = {}

# Create pipelines for each type (2D/3D)
for step in steps_type:
    # Apply threshold only for 2D
    if step == '2D':
        for thr in steps_thr:
            for norm_method in normalization_methods:
                for i, pre_steps in enumerate(pre_steps_permutations):
                    for j, post_steps in enumerate(post_steps_permutations_2d):
                        pipeline_pre = pre_steps_top + [norm_method] + list(pre_steps)
                        pipeline = pipeline_pre + list(post_steps)
                        
                        # Include threshold for 2D pipelines
                        name = f"pipeline_{step}_thr_{thr}_{norm_method}_{i}_{j}"
                        pipelines_dict[name] = {
                            'pipeline': pipeline,
                            'type': step,
                            'threshold': thr
                        }
    else:  # For 3D pipelines, ignore thresholds
        for norm_method in normalization_methods:
            for prob_t in steps_prob_thr:
                for i, pre_steps in enumerate(pre_steps_permutations):
                    for j, post_steps in enumerate(post_steps_permutations_3d):
                        pipeline_pre = pre_steps_top + [norm_method] + list(pre_steps)
                        pipeline = pipeline_pre + list(post_steps)
                        
                        # No threshold for 3D pipelines
                        name = f"pipeline_{step}_prob_{prob_t}_{norm_method}_{i}_{j}"
                        pipelines_dict[name] = {
                            'pipeline': pipeline,
                            'type': step,
                            'prob_threshold': prob_t
                        }

print('Total pipelines:', len(pipelines_dict))

Total pipelines: 800


In [4]:
pipelines_dict

{'pipeline_2D_thr_0.2_norm_percentile_0_0': {'pipeline': ['isotropy',
   'norm_percentile',
   'bilateral',
   'merge_connected_components'],
  'type': '2D',
  'threshold': 0.2},
 'pipeline_2D_thr_0.2_norm_percentile_0_1': {'pipeline': ['isotropy',
   'norm_percentile',
   'bilateral',
   'merge_graph'],
  'type': '2D',
  'threshold': 0.2},
 'pipeline_2D_thr_0.2_norm_percentile_0_2': {'pipeline': ['isotropy',
   'norm_percentile',
   'bilateral',
   'clean_boundaries_opening'],
  'type': '2D',
  'threshold': 0.2},
 'pipeline_2D_thr_0.2_norm_percentile_0_3': {'pipeline': ['isotropy',
   'norm_percentile',
   'bilateral',
   'merge_connected_components',
   'merge_graph'],
  'type': '2D',
  'threshold': 0.2},
 'pipeline_2D_thr_0.2_norm_percentile_0_4': {'pipeline': ['isotropy',
   'norm_percentile',
   'bilateral',
   'merge_connected_components',
   'clean_boundaries_opening'],
  'type': '2D',
  'threshold': 0.2},
 'pipeline_2D_thr_0.2_norm_percentile_0_5': {'pipeline': ['isotropy',
   

In [6]:
model = my_cellpose.load_model('nuclei')
tester = ModelTester(model)

Loading model: nuclei


In [ ]:
bar = LoadingBar(len(pipelines_dict))

for name, config in pipelines_dict.items():
    print(f"\nTesting {name} pipeline")
    
    pipeline = config['pipeline']
    type = config['type']
    thr = config['threshold'] if 'threshold' in config else None
    prob_thr = config['prob_threshold'] if 'prob_threshold' in config else 0.0
    
    tester.run(
        img_path, pipeline,
        type=type, stitch_threshold=thr,
        test_name=name, cellprob_threshold=prob_thr,
        verbose=0
    )
    
    bar.update()
    
bar.end()


Testing pipeline_2D_thr_0.2_norm_percentile_0_0 pipeline


100%|██████████| 19/19 [00:00<00:00, 23.82it/s]


[                                                  ] 0.12%
Testing pipeline_2D_thr_0.2_norm_percentile_0_1 pipeline


100%|██████████| 19/19 [00:00<00:00, 3546.11it/s]


Number of cells after linking: 6
[                                                  ] 0.25%
Testing pipeline_2D_thr_0.2_norm_percentile_0_2 pipeline


...

100%|██████████| 19/19 [00:00<00:00, 3482.58it/s]


Number of cells after linking: 6
[                                                  ] 1.50%
Testing pipeline_2D_thr_0.2_norm_percentile_0_12 pipeline


100%|██████████| 19/19 [00:00<00:00, 4218.50it/s]


Number of cells after linking: 6
[                                                  ] 1.62%
Testing pipeline_2D_thr_0.2_norm_percentile_0_13 pipeline


100%|██████████| 19/19 [00:00<00:00, 3672.77it/s]
